In [ ]:
import pickle
import numpy as np
import mplhep
import awkward
import matplotlib.pyplot as plt

In [ ]:
# physics_process = "ttbar"
# data_mlpf = awkward.Array(pickle.load(open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11834.13/out.pkl", "rb")))
# data_baseline = awkward.Array(pickle.load(open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11834.13_baseline/out.pkl", "rb")))

In [ ]:
physics_process = "singlepi"
data_mlpf = awkward.Array(pickle.load(open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11688.0_mlpf/out.pkl", "rb")))
data_baseline = awkward.Array(pickle.load(open("/home/joosep/reco/mlpf/CMSSW_11_3_0_pre2/11688.0_baseline/out.pkl", "rb")))

In [ ]:
def cms_label(x0=0.12, x1=0.23, x2=0.67, y=0.90):
    plt.figtext(x0, y,'CMS',fontweight='bold', wrap=True, horizontalalignment='left', fontsize=12)
    plt.figtext(x1, y,'Simulation Preliminary', style='italic', wrap=True, horizontalalignment='left', fontsize=10)
    plt.figtext(x2, y,'Run 3 (14 TeV)',  wrap=True, horizontalalignment='left', fontsize=10)
    
physics_process_str = {
    "ttbar": "$\mathrm{t}\overline{\mathrm{t}}$ events",
    "singlepi": "single $\pi^{\pm}$ events",
}
def sample_label(ax, x=0.03, y=0.98):
    plt.text(x, y, physics_process_str[physics_process], va="top", ha="left", size=10, transform=ax.transAxes)


In [ ]:
plt.figure(figsize=(5, 5))
ax = plt.axes()

bins = np.linspace(0, 1000, 61)
plt.hist(awkward.flatten(data_baseline["ak4PFJetsCHS"]["pt"]), bins=bins, histtype="step", lw=2, label="PF");
plt.hist(awkward.flatten(data_mlpf["ak4PFJetsCHS"]["pt"]), bins=bins, histtype="step", lw=2, label="MLPF");
plt.yscale("log")
cms_label()
sample_label(ax, x=0.1)
plt.xlabel("ak4PFJetsCHS $p_T$ [GeV]")
plt.ylabel("Number of jets")
plt.legend(loc="best")

plt.savefig("ak4jet_pt_{}.pdf".format(physics_process), bbox_inches="tight")

In [ ]:
plt.figure(figsize=(5, 5))

bins = np.linspace(0, 2000, 61)
plt.hist(awkward.flatten(data_baseline["ak4PFJetsCHS"]["energy"]), bins=bins, histtype="step", lw=2, label="PF");
plt.hist(awkward.flatten(data_mlpf["ak4PFJetsCHS"]["energy"]), bins=bins, histtype="step", lw=2, label="MLPF");
plt.yscale("log")
cms_label()
sample_label(ax, x=0.1)

plt.xlabel("ak4PFJetsCHS $E$ [GeV]")
plt.ylabel("Number of jets")
plt.legend(loc="best")

plt.savefig("ak4jet_energy_{}.pdf".format(physics_process), bbox_inches="tight")

In [ ]:
plt.figure(figsize=(5, 5))

bins = np.linspace(-6, 6, 41)
plt.hist(awkward.flatten(data_baseline["ak4PFJetsCHS"]["eta"]), bins=bins, histtype="step", lw=2, label="PF");
plt.hist(awkward.flatten(data_mlpf["ak4PFJetsCHS"]["eta"]), bins=bins, histtype="step", lw=2, label="MLPF");
#plt.yscale("log")
cms_label()
sample_label(ax)

plt.xlabel("ak4PFJetsCHS $\eta$")
plt.ylabel("Number of jets")
plt.legend(loc="best")

plt.savefig("ak4jet_eta_{}.pdf".format(physics_process), bbox_inches="tight")

In [ ]:
color_map = {
    1: "red",
    2: "blue",
    11: "orange",
    22: "cyan",
    13: "purple",
    130: "green",
    211: "black"
}

In [ ]:
def draw_event(iev):
    pt_0 = data_mlpf["particleFlow"]["pt"][iev]
    energy_0 = data_mlpf["particleFlow"]["energy"][iev]
    eta_0 = data_mlpf["particleFlow"]["eta"][iev]
    phi_0 = data_mlpf["particleFlow"]["phi"][iev]
    pdgid_0 = np.abs(data_mlpf["particleFlow"]["pdgId"][iev])
    
    pt_1 = data_baseline["particleFlow"]["pt"][iev]
    energy_1 = data_baseline["particleFlow"]["energy"][iev]
    eta_1 = data_baseline["particleFlow"]["eta"][iev]
    phi_1 = data_baseline["particleFlow"]["phi"][iev]
    pdgid_1 = np.abs(data_baseline["particleFlow"]["pdgId"][iev])
    
    plt.figure(figsize=(5, 5))
    plt.scatter(eta_0, phi_0, marker=".", s=energy_0, c=[color_map[p] for p in pdgid_0], alpha=0.6)
    # plt.scatter(
    #     data_mlpf["ak4PFJetsCHS"]["eta"][iev],
    #     data_mlpf["ak4PFJetsCHS"]["phi"][iev],
    #     s=data_mlpf["ak4PFJetsCHS"]["energy"][iev], color="gray", alpha=0.3
    # )
    cms_label()
    sample_label(ax)
    plt.xlim(-6,6)
    plt.ylim(-4,4)
    plt.xlabel("PFCandidate $\eta$")
    plt.ylabel("PFCandidate $\phi$")
    plt.title("MLPF (trained on PF)", y=1.05)
    plt.savefig("event_mlpf_{}_iev{}.pdf".format(physics_process, iev), bbox_inches="tight")
    
    plt.figure(figsize=(5, 5))
    plt.scatter(eta_1, phi_1, marker=".", s=energy_1, c=[color_map[p] for p in pdgid_1], alpha=0.6)
    # plt.scatter(
    #     data_baseline["ak4PFJetsCHS"]["eta"][iev],
    #     data_baseline["ak4PFJetsCHS"]["phi"][iev],
    #     s=data_baseline["ak4PFJetsCHS"]["energy"][iev], color="gray", alpha=0.3
    # )
    cms_label()
    sample_label(ax)
    plt.xlim(-6,6)
    plt.ylim(-4,4)
    plt.xlabel("PFCandidate $\eta$")
    plt.ylabel("PFCandidate $\phi$")
    plt.title("Standard PF", y=1.05)
    plt.savefig("event_pf_{}_iev{}.pdf".format(physics_process, iev), bbox_inches="tight")

In [ ]:
draw_event(0)

In [ ]:
draw_event(1)

In [ ]:
draw_event(2)